In [1]:
import numpy as np
import pandas as pd
import math
import datetime as dt
import datapackage
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import multinomial
from typing import List
import seaborn as sns

## system characterisation ##

class State: 
    def __init__(self,mean,eta,lambda_switch,sigma):
        self.m = mean
        self.e = eta 
        self.l = lambda_switch
        self.s = sigma

M = 2
N = 2

state_1 = State(50,0.44,0.26,0.23)

def Profit(E,P,vc,fc,pi_tax,c_tax):
    rev = E*P
    cost = E*(vc+c_tax) + fc 
    profit = (rev - cost)*(1 - pi_tax)
    return(profit)

T = 30
E = 2500000
r = 0.05 
E = 5
vc = 10
fc = 0
pi_tax = 0.2
c_tax = 5


## node construction ##

class node:
    def __init__(self,value,temp):
        self.value = v
        self.temp = t

## finite difference parameter definition ##

## parameter defn for P_i in i = 2,...,i_max-1 ##

def difference(P,p_step,sigma,eta,mean):
    a_i_central = (sigma**2)*(P**2)/((p_step*2)*p_step) - eta*(mean-P)/(p_step*2)
    b_i_central = (sigma**2)*(P**2)/((p_step*2)*p_step) - eta*(mean-P)/(p_step*2)
    a_i_forward = (sigma**2)*(P**2)/((p_step*2)*p_step)
    b_i_forward = (sigma**2)*(P**2)/((p_step*2)*p_step) - eta*(mean-P)/(p_step*2)
    a_i_backward = (sigma**2)*(P**2)/((p_step*2)*p_step) - eta*(mean-P)/(p_step*2)
    b_i_backward = (sigma**2)*(P**2)/((p_step*2)*p_step)
    if {a_i_central and b_i_central >= 0}:
        return [a_i_central,b_i_central]
    elif b_i_forward >= 0:
        return [a_i_forward,b_i_forward]
    else: 
        return [a_i_backward,b_i_backward]
    
## parameter defn for P_i in i = 1 ##
    
def difference_min(P,p_step,eta,mean):
    a_i = 0
    b_i = eta*(mean-P)/(p_step)
    return [a_i,b_i]

## parameter defn for P_i in i = i_max ##

def difference_max(P,p_step,eta,mean):
    a_i = eta*(mean - P)/(p_step)
    b_i = 0
    return [a_i,b_i]

In [2]:
## grid parameters ##

p_step = 1
s0_step = 5000

p_1 = 20
p_max = 100

s0_1 = 5000
s0_max = 20000

## grid construction ##

P_grid = list(range(p_1,p_max,p_step))
s0_grid = list(range(s0_1,s0_max,s0_step))

## subnode lattice construction to describe temporary values

temp_node = np.full((len(P_grid),len(s0_grid),M,N),0)

Lv = np.zeros((len(P_grid),len(s0_grid),M,N))

print(Profit(E,60,vc,fc,pi_tax,c_tax))

180.0


In [3]:
for q in range(4):
        
        ## Quarterly profit at each node ## 
        
        for i in range(len(P_grid)):
            for k in range(len(s0_grid)):
                for m in range(M):
                    for n in range(N):
                        if n == 1:
                            temp_node[i,k,m,n] = temp_node[i,k,m,n] + Profit(E,P_grid[i],vc,fc,pi_tax,c_tax)
                        else:
                            temp_node[i,k,m,n] = temp_node[i,k,m,n] - fc
        
        ## Lagrange Differential at each node ##
        
## then we loop over Lagrange differential for boundary nodes ##                

        for k in range(len(s0_grid)):
            for m in range(M):
                for n in range(N):
                    diff = difference_min(P_grid[0],p_step,state_1.e,state_1.m)
                    Lv[0,k,m,n] = diff[1]*temp_node[1,k,m,n] - (diff[0]+diff[1]+r)*temp_node[0,k,m,n]

        for k in range(len(s0_grid)):
            for m in range(M):
                for n in range(N):
                    diff = difference_max(len(P_grid) - 1,p_step,state_1.e,state_1.m)
                    Lv[len(P_grid) - 1,k,m,n] = diff[0]*temp_node[len(P_grid) - 2,k,m,n]  - (diff[0]+diff[1]+r)*temp_node[len(P_grid) - 1,k,m,n]

## then we loop over lagrange differential for nodes inside the boundary ##
    
        for i in range(1,len(P_grid) - 1):
            for k in range(len(s0_grid)):
                for m in range(M):
                    for n in range(N):
                        diff = difference(P_grid[i],p_step,state_1.s,state_1.e,state_1.m)
                        Lv[i,k,m,n] = diff[0]*temp_node[i - 1,k,m,n] + diff[1]*temp_node[i + 1,k,m,n] - (diff[0]+diff[1]+r)*temp_node[i,k,m,n]

## then we apply Lv to temporary node values ##
        for i in range(len(P_grid)):
            for k in range(len(s0_grid)):
                for m in range(M):
                    for n in range(N):
                         temp_node[i,k,m,n] = temp_node[i,k,m,n] + Lv[i,k,m,n]  

OverflowError: Python int too large to convert to C long

In [4]:
print(Lv)

[[[[ 0.00000000e+00 -2.12190300e+05]
   [ 0.00000000e+00 -2.12190300e+05]]

  [[ 0.00000000e+00 -2.12190300e+05]
   [ 0.00000000e+00 -2.12190300e+05]]

  [[ 0.00000000e+00 -2.12190300e+05]
   [ 0.00000000e+00 -2.12190300e+05]]]


 [[[ 0.00000000e+00  1.24246647e+05]
   [ 0.00000000e+00  1.24246647e+05]]

  [[ 0.00000000e+00  1.24246647e+05]
   [ 0.00000000e+00  1.24246647e+05]]

  [[ 0.00000000e+00  1.24246647e+05]
   [ 0.00000000e+00  1.24246647e+05]]]


 [[[ 0.00000000e+00 -6.15743008e+04]
   [ 0.00000000e+00 -6.15743008e+04]]

  [[ 0.00000000e+00 -6.15743008e+04]
   [ 0.00000000e+00 -6.15743008e+04]]

  [[ 0.00000000e+00 -6.15743008e+04]
   [ 0.00000000e+00 -6.15743008e+04]]]


 [[[ 0.00000000e+00  1.39640568e+04]
   [ 0.00000000e+00  1.39640568e+04]]

  [[ 0.00000000e+00  1.39640568e+04]
   [ 0.00000000e+00  1.39640568e+04]]

  [[ 0.00000000e+00  1.39640568e+04]
   [ 0.00000000e+00  1.39640568e+04]]]


 [[[ 0.00000000e+00  5.40318360e+03]
   [ 0.00000000e+00  5.40318360e+03]]

  [[